In [6]:
# try langchain pdf loader + vector db
# https://medium.com/@cch.chichieh/rag%E5%AF%A6%E4%BD%9C%E6%95%99%E5%AD%B8-langchain-llama2-%E5%89%B5%E9%80%A0%E4%BD%A0%E7%9A%84%E5%80%8B%E4%BA%BAllm-d6838febf8c4

In [ ]:
#pip install -r requirements.txt
#pip install langchain-openai for using azureopenai
# %pip install chromadb

In [ ]:
import os
from openai import AzureOpenAI
# get config from aoai_config.yaml

%run env_config_setting.ipynb

api_version = openai.api_version
api_key = openai.api_key
api_base = openai.api_base

print('openi config: ', deployment_name, embedding_name, api_key, api_base, api_version)
print('search config: ', search_service_name, search_key, search_endpoint)

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./pdfs/eda_llm_future.pdf")
PDF_data = loader.load()

#PyMuPDFLoader without table and image elements 
#圖片文字跟table 欄位 內容都有x擷取出來 但是沒有圖片資訊
# has \n for each line
PDF_data


[Document(metadata={'source': './pdfs/eda_llm_future.pdf', 'page': 0}, page_content='Large Language Models for EDA: Future or Mirage?\nZhuolun He\nThe Chinese University of Hong Kong\nHong Kong SARBei Yu\nThe Chinese University of Hong Kong\nHong Kong SAR\nABSTRACT\nIn this paper, we explore the burgeoning intersection of Large Lan-\nguage Models (LLMs) and Electronic Design Automation (EDA). We\ncritically assess whether LLMs represent a transformative future\nfor EDA or merely a fleeting mirage. By analyzing current advance-\nments, challenges, and potential applications, we dissect how LLMs\ncan revolutionize EDA processes like design, verification, and op-\ntimization. Furthermore, we contemplate the ethical implications\nand feasibility of integrating these models into EDA workflows. Ulti-\nmately, this paper aims to provide a comprehensive, evidence-based\nperspective on the role of LLMs in shaping the future of EDA.\nACM Reference Format:\nZhuolun He and Bei Yu. 2024. Large Lang

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_overlap 設5 但是看起來沒有重疊的部分 ?
# chunk_size 設100 但針對每個切完的文字放到word 透過「字數統計」。也有長度是超過字元數100的 ?

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)
#all_splits


In [6]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    deployment=embedding_name,
    azure_endpoint=api_base,
    api_key=api_key,
    chunk_size=1)


In [7]:
#if using different embedding model, need saved other db file

# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain.vectorstores import Chroma

#this will create a folder [db] and add a new file chroma.sqlite3

persist_directory = 'chroma_langchain_aoai_retrieval_try'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()

⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.
/var/folders/n7/1tk7cfpx0_31x9yrrl6skynh0000gn/T/ipykernel_15034/3739428637.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [8]:
def search_documents_by_chroma(index=persist_directory, query="", limit=5):
    vectordb = Chroma(persist_directory=index, embedding_function=embedding)
    docs = vectordb.similarity_search(query, k=limit)
    return docs


In [9]:
search_documents_by_chroma(query="what is multi-stage amplifiers")

/var/folders/n7/1tk7cfpx0_31x9yrrl6skynh0000gn/T/ipykernel_15034/1162148580.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=index, embedding_function=embedding)


[Document(metadata={'author': '', 'creationDate': "D:20240108184338+00'00", 'creator': 'LaTeX with hyperref package', 'file_path': '/Users/lizylin/Downloads/[2023] LADAC Large Language Model-driven Auto-Designer for Analog Circuits.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20240108184338+00'00", 'page': 5, 'producer': 'Ruby CombinePDF 1.0.21 Library', 'source': '/Users/lizylin/Downloads/[2023] LADAC Large Language Model-driven Auto-Designer for Analog Circuits.pdf', 'subject': '', 'title': '', 'total_pages': 10, 'trapped': ''}, page_content='ually refined knowledge about multi-stage amplifiers, ring'),
 Document(metadata={'author': '', 'creationDate': "D:20240108184338+00'00", 'creator': 'LaTeX with hyperref package', 'file_path': '/Users/lizylin/Downloads/[2023] LADAC Large Language Model-driven Auto-Designer for Analog Circuits.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20240108184338+00'00", 'page': 6, 'producer': 'Ruby CombinePDF 1.0.21 Library', 'sourc

In [11]:
from langchain.chat_models import AzureChatOpenAI

client = AzureChatOpenAI(
    deployment_name=deployment_name,
    temperature=0,
    api_key=api_key,
    azure_endpoint=api_base,
    openai_api_version=api_version)


In [12]:
from langchain.chains import RetrievalQA

retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=client, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [13]:
qa.invoke("what is LADAC?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is LADAC?',
 'result': 'LADAC is a new circuit design process that utilizes the LLM (Large Language Model) agent approach. It is guided by a local knowledge library and aims to demonstrate practicality and potential in circuit design.'}

In [14]:
chat_history = []

q1 = "what part is the most important"
result = qa({"query": q1, "chat_history": chat_history})
result

/var/folders/n7/1tk7cfpx0_31x9yrrl6skynh0000gn/T/ipykernel_15034/2444668796.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": q1, "chat_history": chat_history})




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what part is the most important',
 'chat_history': [],
 'result': "I don't have enough context to determine which part is the most important. The provided text seems to be fragmented and lacks specific details about the subject matter. If you can provide more context or clarify your question, I might be able to help better."}

In [15]:
chat_history.append((q1, result['result']))

q1 = "what part is the most important in LADAC?"
result = qa({"query": q1, "chat_history": chat_history})
result



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what part is the most important in LADAC?',
 'chat_history': [('what part is the most important',
   "I don't have enough context to determine which part is the most important. The provided text seems to be fragmented and lacks specific details about the subject matter. If you can provide more context or clarify your question, I might be able to help better.")],
 'result': 'The central component of LADAC is considered the most important part.'}

In [ ]:
#====== use promptTemplate and ConversationalRetrievalChain

In [16]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

crqa = ConversationalRetrievalChain.from_llm(llm=client,
                                           retriever=retriever,
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=True)

In [17]:
crchat_history = []
query = "what is LADAC?"
crresult = crqa({"question": query, "chat_history": crchat_history})
print(crresult["answer"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
V. CONCLUSION
In this study, we have presented LADAC, which is a new

circuit design process by using the LLM agent approach.
III. OVERVIEW OF LADAC

Guided by the local knowledge library, LADAC achieves
TABLE I

• Demonstrating the practicality and potential of LADAC
Human: what is LADAC?

> Finished chain.

> Finished chain.
LADAC is a new circuit design process that utilizes the LLM (Large Language Model) agent approach.
